In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Path to your CSV file on Google Drive
csv_file_path = '/content/drive/MyDrive/Obama(p) 2009-2016 - BarackObama.csv'

# Read the CSV file using pandas
data = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
data.head()


,Unnamed: 0,date,id,link,retweet,text,author
0,0,20h20 hours ago,7.869827e+17,/BarackObama/status/786982739517943808,False,Denying climate change is dangerous. Join @OFA...,BarackObama
1,1,18h18 hours ago,7.870101e+17,/BarackObama/status/787010142378332160,False,The American Bar Association gave Judge Garlan...,BarackObama
2,2,16h16 hours ago,7.870398e+17,/BarackObama/status/787039774330748928,False,We need a fully functional Supreme Court. Edit...,BarackObama
3,3,21h21 hours ago,7.869644e+17,/BarackObama/status/786964419905523712,False,"Cynics, take note: When we #ActOnClimate, we b...",BarackObama
4,4,Oct 13,7.866806e+17,/BarackObama/status/786680553617629185,False,"""That’s how we will overcome the challenges we...",BarackObama


In [4]:
data.drop("Unnamed: 0",axis=1,inplace=True)

In [5]:
data.columns

Index(['date', 'id', 'link', 'retweet', 'text', 'author'], dtype='object')

In [6]:
# List of columns to drop
columns_to_drop = ['id', 'link']

In [7]:
# Drop the specified columns
df = data.drop(columns=columns_to_drop)

In [8]:
df.head()

,date,retweet,text,author
0,20h20 hours ago,False,Denying climate change is dangerous. Join @OFA...,BarackObama
1,18h18 hours ago,False,The American Bar Association gave Judge Garlan...,BarackObama
2,16h16 hours ago,False,We need a fully functional Supreme Court. Edit...,BarackObama
3,21h21 hours ago,False,"Cynics, take note: When we #ActOnClimate, we b...",BarackObama
4,Oct 13,False,"""That’s how we will overcome the challenges we...",BarackObama


In [9]:
df.value_counts("retweet")

retweet
False    6684
dtype: int64

In [10]:
df.loc[df.isnull().any(axis=1)]

,date,retweet,text,author


In [11]:
!pip install advertools

In [12]:
%config InlineBackend.figure_format = 'retina' # high resolution plotting
import matplotlib.pyplot as plt
import pandas as pd
import advertools as adv
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 280)
adv.__version__

'0.13.5'

# Mentions

In [13]:
mention_summary = adv.extract_mentions(df['text'])
mention_summary.keys()

dict_keys(['mentions', 'mentions_flat', 'mention_counts', 'mention_freq', 'top_mentions', 'overview'])

In [14]:
df['user_id']=mention_summary['mentions']

In [15]:
df.head()

,date,retweet,text,author,user_id
0,20h20 hours ago,False,Denying climate change is dangerous. Join @OFA supporters in standing up for bold action now: http://ofa.bo/2dZNTRx #ActOnClimate,BarackObama,[@ofa]
1,18h18 hours ago,False,The American Bar Association gave Judge Garland its highest rating. Senate leaders: #DoYourJob and fill the #SCOTUS vacancy.pic.twitter.com/4yseqnOBX4,BarackObama,[]
2,16h16 hours ago,False,We need a fully functional Supreme Court. Editorial boards across the country agree: http://ofa.bo/2dCG7KU #DoYourJob,BarackObama,[]
3,21h21 hours ago,False,"Cynics, take note: When we #ActOnClimate, we build a stronger future.pic.twitter.com/wHHMdo2ro6",BarackObama,[]
4,Oct 13,False,"""That’s how we will overcome the challenges we face: by unleashing the power of all of us for all of us."" Read more:http://ofa.bo/2dNlZZy",BarackObama,[]


# Sentiment Analysis

In [16]:
df1=pd.read_csv("/content/drive/MyDrive/@mentions_list(obama_P).csv")

In [17]:
df1.head()

,user_id,frequency
0,@michelleobama,29
1,@ofa,28
2,@whitehouse,24
3,@obama2012,13
4,@vp,12


In [18]:
pip install vaderSentiment

In [19]:
#sentiment analysis for the period before presidency
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from statistics import mode

# Create a SentimentIntensityAnalyzer object
analyzer = SentimentIntensityAnalyzer()

# Function to get sentiment label
def get_sentiment_label(score):
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

# Perform sentiment analysis and add sentiment column
df['Sentiment_Scores'] = df['text'].apply(lambda text: analyzer.polarity_scores(text))
df['Sentiment'] = df['Sentiment_Scores'].apply(lambda scores: get_sentiment_label(scores['compound']))

# Assuming you have two dataframes df1 and df2
# Iterate through df2
for index, row in df1.iterrows():
    userid = row['user_id']

    # Filter df to get tweets containing the userid
    matching_tweets = df[df['text'].str.contains(userid, case=False)]

    if not matching_tweets.empty:
        # Calculate sentiment scores for matching tweets
        matching_tweets['sentiment_score'] = matching_tweets['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

        # Get sentiment labels using the provided function
        matching_tweets['sentiment_label'] = matching_tweets['sentiment_score'].apply(get_sentiment_label)

        # Calculate the mode sentiment
        mode_sentiment = mode(matching_tweets['sentiment_label'])

        # Add mode sentiment to df1
        df1.at[index, 'sentiment_during_presidency'] = mode_sentiment

# Save the updated df2 with sentiment information
df1.to_csv('Obama_sentiment_analysis(P).csv', index=False)


<ipython-input-19-cadbff698da4>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_tweets['sentiment_score'] = matching_tweets['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
<ipython-input-19-cadbff698da4>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_tweets['sentiment_label'] = matching_tweets['sentiment_score'].apply(get_sentiment_label)
<ipython-input-19-cadbff698da4>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [20]:
df.head()

,date,retweet,text,author,user_id,Sentiment_Scores,Sentiment
0,20h20 hours ago,False,Denying climate change is dangerous. Join @OFA supporters in standing up for bold action now: http://ofa.bo/2dZNTRx #ActOnClimate,BarackObama,[@ofa],"{'neg': 0.218, 'neu': 0.476, 'pos': 0.306, 'compound': 0.296}",Positive
1,18h18 hours ago,False,The American Bar Association gave Judge Garland its highest rating. Senate leaders: #DoYourJob and fill the #SCOTUS vacancy.pic.twitter.com/4yseqnOBX4,BarackObama,[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",Neutral
2,16h16 hours ago,False,We need a fully functional Supreme Court. Editorial boards across the country agree: http://ofa.bo/2dCG7KU #DoYourJob,BarackObama,[],"{'neg': 0.0, 'neu': 0.671, 'pos': 0.329, 'compound': 0.749}",Positive
3,21h21 hours ago,False,"Cynics, take note: When we #ActOnClimate, we build a stronger future.pic.twitter.com/wHHMdo2ro6",BarackObama,[],"{'neg': 0.101, 'neu': 0.698, 'pos': 0.202, 'compound': 0.3182}",Positive
4,Oct 13,False,"""That’s how we will overcome the challenges we face: by unleashing the power of all of us for all of us."" Read more:http://ofa.bo/2dNlZZy",BarackObama,[],"{'neg': 0.0, 'neu': 0.944, 'pos': 0.056, 'compound': 0.0772}",Positive


In [21]:
df.shape

(6684, 7)

In [22]:
df1.head()

,user_id,frequency,sentiment_during_presidency
0,@michelleobama,29,Positive
1,@ofa,28,Positive
2,@whitehouse,24,Positive
3,@obama2012,13,Neutral
4,@vp,12,Positive


In [23]:
df1.shape

(257, 3)